In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise
import math

In [2]:
background = None
accumulated_weight_alpha = 0.5

roi_left = 50
roi_right = 350
roi_top = 20
roi_bottom = 320

In [3]:
def calculate_accumulated_weight(frame, accumulated_weight_alpha):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight_alpha)


In [4]:
def segment_finder(frame, threshold_min = 25):
    global background
    diff = cv2.absdiff(background.astype('uint8'), frame)
    _, threshold = cv2.threshold(diff, threshold_min, 255,  cv2.THRESH_BINARY)
    contour, _ = cv2.findContours(threshold.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contour)== 0:
        return None
    hand_segment = max(contour, key= cv2.contourArea)

    return(threshold, hand_segment)

In [5]:
def count_fingers(threshold, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])

    cx = (left[0] + right[0]) // 2
    cy = (bottom[0] + top[0]) // 2

    distance = pairwise.euclidean_distances(X= [[cx, cy]], Y= [top, bottom, left, right])
    max_distance = distance.max()
    radius = int(0.85 * max_distance)
    circumference = math.pi * 2 * radius

    circular_roi = np.zeros((threshold.shape[0], threshold.shape[1]), dtype='uint8')
    cv2.circle(circular_roi, (cx, cy), radius, (0,0,255), 5)

    circular_roi = cv2.bitwise_and(threshold, threshold, mask= circular_roi)
    contour, _ = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    count = 0

    for cnt in contour:
        (x,y,w,h) = cv2.boundingRect(cnt)
        out_of_wrist = ((1 + 0.25) * cy) > ((y+h))
        limit_points= ((circumference * 0.25) > cnt.shape[0])

        if limit_points and out_of_wrist :
            count += 1

    return count

In [6]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    _, frame = cam.read()
    if frame is not None : 
        frame_copy = frame.copy()
    if frame is None:
        continue
    roi = frame[roi_top : roi_bottom, roi_left : roi_right]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)


    gray = cv2.GaussianBlur(gray, (7,7), 0)

    if num_frames < 60:
        calculate_accumulated_weight(gray, accumulated_weight_alpha)
        if num_frames <= 59:
            cv2.putText(frame_copy, 'Warning', (200,300),  cv2.FONT_HERSHEY_COMPLEX,1, (0,0,255), 4)
            cv2.imshow("FingerCount", frame_copy)
    
    else:
        hand = segment_finder(gray)

        if hand is None:
            continue

        thresholded, handsegment = hand

        handsegment = handsegment + (roi_left, roi_top)
        cv2.drawContours(frame_copy, [handsegment], -1, (255, 0, 0), 5)

        fingers = count_fingers(thresholded, handsegment)

        cv2.putText(frame_copy, str(fingers), (350,350), cv2.FONT_HERSHEY_COMPLEX,1, (0,255,0), 10)
        cv2.imshow("Thresholded", thresholded)

    cv2.rectangle(frame_copy, (roi_left, roi_top),(roi_right, roi_bottom), (0,0,255), 10)
    num_frames += 1
    cv2.imshow("FingerCount", frame_copy)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q') or cv2.getWindowProperty("FingerCount", cv2.WND_PROP_VISIBLE) < 1:
        break


cam.release()
cv2.destroyAllWindows()

